<a href="https://colab.research.google.com/github/lunaB/AI-Experiment/blob/master/%EB%87%8C%ED%8C%8C_%EC%98%81%EC%83%81%EB%AC%BC_%EC%8B%9C%EC%B2%AD_%EC%9E%A5%EB%A5%B4_%EB%B6%84%EC%84%9D%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Kaggle 데이터 로드

In [5]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d birdy654/eeg-brainwave-dataset-feeling-emotions
!unzip eeg-brainwave-dataset-feeling-emotions.zip

eeg-brainwave-dataset-feeling-emotions.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  eeg-brainwave-dataset-feeling-emotions.zip
replace emotions.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


## 데이터 기본정보

- 2548 특징값과 1개의 결과값을 가지고 있는 데이터
- 분류 라벨은 3가지
- 행의 갯수 2132개
- 결측치 없음


In [6]:
import pandas as pd

data = pd.read_csv('emotions.csv')
data.head()

,# mean_0_a,mean_1_a,mean_2_a,mean_3_a,mean_4_a,mean_d_0_a,mean_d_1_a,mean_d_2_a,mean_d_3_a,mean_d_4_a,mean_d_0_a2,mean_d_1_a2,mean_d_2_a2,mean_d_3_a2,mean_d_4_a2,mean_d_5_a,mean_d_6_a,mean_d_7_a,mean_d_8_a,mean_d_9_a,mean_d_10_a,mean_d_11_a,mean_d_12_a,mean_d_13_a,mean_d_14_a,mean_d_15_a,mean_d_16_a,mean_d_17_a,mean_d_18_a,mean_d_19_a,mean_d_20_a,mean_d_21_a,mean_d_22_a,mean_d_23_a,mean_d_24_a,mean_d_25_a,mean_d_26_a,mean_d_27_a,mean_d_28_a,mean_d_29_a,...,fft_711_b,fft_712_b,fft_713_b,fft_714_b,fft_715_b,fft_716_b,fft_717_b,fft_718_b,fft_719_b,fft_720_b,fft_721_b,fft_722_b,fft_723_b,fft_724_b,fft_725_b,fft_726_b,fft_727_b,fft_728_b,fft_729_b,fft_730_b,fft_731_b,fft_732_b,fft_733_b,fft_734_b,fft_735_b,fft_736_b,fft_737_b,fft_738_b,fft_739_b,fft_740_b,fft_741_b,fft_742_b,fft_743_b,fft_744_b,fft_745_b,fft_746_b,fft_747_b,fft_748_b,fft_749_b,label
0,4.62,30.3,-356.0,15.6,26.3,1.070,0.411,-15.70,2.06,3.15,2.15,29.5,-353.0,14.40,21.5,5.98,30.7,-343.0,14.7,27.9,3.17,32.2,-368.0,15.9,36.4,7.08,28.8,-359.0,17.3,19.6,-3.8300,-1.230,-10.80000,-0.363,-6.41,-1.03,-2.780,14.60,-1.540,-14.900000,...,30.90,-24.4,-24.4,30.90,-442.0,388.0,-197.0,-197.0,388.0,-564.0,500.00,-245.00,-245.00,500.00,-88.8,214.0,-88.8,-88.8,214.0,-606.0,509.000,-261.0,-261.0,509.000,-399.0,374.0,-185.00,-185.00,374.0,74.3,23.5,20.3,20.3,23.5,-215.0,280.00,-162.00,-162.00,280.00,NEGATIVE
1,28.80,33.1,32.0,25.8,22.8,6.550,1.680,2.88,3.83,-4.82,25.60,32.8,29.6,21.50,17.4,25.50,31.7,31.5,26.2,32.9,31.80,33.1,33.2,28.5,26.8,32.40,34.7,33.8,27.0,14.2,0.0342,1.100,-1.87000,-4.690,-15.40,-6.22,-0.328,-3.53,-6.980,-9.370000,...,-5.99,-19.1,-19.1,-5.99,163.0,-11.0,-10.7,-10.7,-11.0,117.0,-3.93,17.90,17.90,-3.93,112.0,-13.9,25.5,25.5,-13.9,225.0,-0.968,-27.7,-27.7,-0.968,97.4,-19.0,40.70,40.70,-19.0,130.0,-23.3,-21.8,-21.8,-23.3,182.0,2.57,-31.60,-31.60,2.57,NEUTRAL
2,8.90,29.4,-416.0,16.7,23.7,79.900,3.360,90.20,89.90,2.03,7.75,30.1,-441.0,9.89,25.3,-68.90,25.3,-481.0,-65.4,20.0,79.80,31.0,-408.0,91.9,29.5,18.80,31.1,-335.0,32.2,19.9,76.6000,4.850,39.90000,75.300,5.27,-72.00,-0.843,-33.00,-82.000,-4.200000,...,585.00,-285.0,-285.0,585.00,-94.3,183.0,-110.0,-110.0,183.0,-133.0,200.00,-72.00,-72.00,200.00,-356.0,295.0,-158.0,-158.0,295.0,-251.0,255.000,-95.7,-95.7,255.000,-177.0,159.0,-48.80,-48.80,159.0,-534.0,462.0,-233.0,-233.0,462.0,-267.0,281.00,-148.00,-148.00,281.00,POSITIVE
3,14.90,31.6,-143.0,19.8,24.3,-0.584,-0.284,8.82,2.30,-1.97,17.30,32.0,-148.0,20.40,22.8,13.20,31.5,-147.0,16.9,27.7,15.70,30.7,-142.0,20.7,22.8,13.60,32.2,-135.0,21.2,23.8,4.1500,0.556,-0.35400,3.460,-4.96,1.63,1.330,-5.83,-0.298,0.000777,...,249.00,-146.0,-146.0,249.00,359.0,-146.0,13.7,13.7,-146.0,119.0,-7.64,-7.17,-7.17,-7.64,-296.0,316.0,-218.0,-218.0,316.0,322.0,-113.000,38.4,38.4,-113.000,245.0,-61.9,-5.08,-5.08,-61.9,-183.0,299.0,-243.0,-243.0,299.0,132.0,-12.40,9.53,9.53,-12.40,POSITIVE
4,28.30,31.3,45.2,27.3,24.5,34.800,-5.790,3.06,41.40,5.52,26.10,34.3,43.7,23.70,20.6,-3.87,34.1,43.7,-10.0,22.9,59.40,26.7,60.3,64.7,26.9,32.30,30.1,33.6,31.6,27.7,30.0000,0.276,-0.00677,33.700,-2.36,-33.30,7.640,-16.60,-41.100,-6.290000,...,18.30,-11.9,-11.9,18.30,159.0,-18.2,-25.6,-25.6,-18.2,194.0,32.70,-54.50,-54.50,32.70,154.0,26.4,-56.4,-56.4,26.4,252.0,9.940,-37.4,-37.4,9.940,172.0,15.9,-7.25,-7.25,15.9,114.0,12.0,38.1,38.1,12.0,119.0,-17.60,23.90,23.90,-17.60,NEUTRAL


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2132 entries, 0 to 2131
Columns: 2549 entries, # mean_0_a to label
dtypes: float64(2548), object(1)
memory usage: 41.5+ MB


In [8]:
print('결측치 {}개'.format(data.isnull().sum().sum()))

결측치 0개


In [0]:
y_label = {
    'NEUTRAL': 0,
    'POSITIVE': 1,
    'NEGATIVE': 2
}

data['label'] = data['label'].apply(lambda x: y_label[x])

In [0]:
import matplotlib.pyplot as plt

# xy값 분리
x_data = data.loc[:,'# mean_0_a':'fft_749_b']
y_data = data.loc[:, 'label']

## 데이터 가공

In [11]:
xxyy = pd.concat((x_data.copy(), y_data.copy()), axis=1).sample(frac=1).reset_index(drop=True)
xxyy.head()

,# mean_0_a,mean_1_a,mean_2_a,mean_3_a,mean_4_a,mean_d_0_a,mean_d_1_a,mean_d_2_a,mean_d_3_a,mean_d_4_a,mean_d_0_a2,mean_d_1_a2,mean_d_2_a2,mean_d_3_a2,mean_d_4_a2,mean_d_5_a,mean_d_6_a,mean_d_7_a,mean_d_8_a,mean_d_9_a,mean_d_10_a,mean_d_11_a,mean_d_12_a,mean_d_13_a,mean_d_14_a,mean_d_15_a,mean_d_16_a,mean_d_17_a,mean_d_18_a,mean_d_19_a,mean_d_20_a,mean_d_21_a,mean_d_22_a,mean_d_23_a,mean_d_24_a,mean_d_25_a,mean_d_26_a,mean_d_27_a,mean_d_28_a,mean_d_29_a,...,fft_711_b,fft_712_b,fft_713_b,fft_714_b,fft_715_b,fft_716_b,fft_717_b,fft_718_b,fft_719_b,fft_720_b,fft_721_b,fft_722_b,fft_723_b,fft_724_b,fft_725_b,fft_726_b,fft_727_b,fft_728_b,fft_729_b,fft_730_b,fft_731_b,fft_732_b,fft_733_b,fft_734_b,fft_735_b,fft_736_b,fft_737_b,fft_738_b,fft_739_b,fft_740_b,fft_741_b,fft_742_b,fft_743_b,fft_744_b,fft_745_b,fft_746_b,fft_747_b,fft_748_b,fft_749_b,label
0,-21.4,27.6,13.2,-34.50,22.2,-44.200,-1.13,16.90,-42.100,2.710,0.93,29.1,4.52,-14.2,24.0,0.495,27.3,5.03,-12.700,17.8,-5.72,26.1,-0.793,-13.6,21.1,-80.2,28.0,43.5,-96.40,26.0,0.434,1.800,-0.518,-1.510,6.21,6.650,3.030,5.310,-0.636,2.90,...,38.40,-29.6,-29.6,38.40,-68.2,15.00,-40.90,-40.90,15.00,52.7,48.00,8.490,8.490,48.00,13.3,25.50,-13.8,-13.8,25.50,68.2,17.40,-104.00,-104.00,17.40,28.2,76.80,13.30,13.30,76.80,33.0,42.70,-51.00,-51.00,42.70,45.5,31.8,-72.9,-72.9,31.8,1
1,13.9,31.4,-167.0,13.70,31.7,-4.590,-2.78,-1.05,-4.820,0.906,14.00,31.4,-161.00,13.8,32.7,18.300,34.1,-172.00,18.300,29.9,12.50,31.9,-177.000,14.1,35.8,10.8,28.1,-158.0,8.46,28.7,-4.290,-2.690,11.900,-4.520,2.76,1.530,-0.525,16.700,-0.321,-3.10,...,281.00,-152.0,-152.0,281.00,334.0,-89.70,9.66,9.66,-89.70,-290.0,294.00,-193.000,-193.000,294.00,-291.0,317.00,-129.0,-129.0,317.00,263.0,-31.90,-8.04,-8.04,-31.90,-471.0,451.00,-253.00,-253.00,451.00,-274.0,293.00,-129.00,-129.00,293.00,153.0,-35.4,53.6,53.6,-35.4,2
2,28.3,31.0,30.4,25.50,27.8,-1.740,-1.40,2.01,3.310,8.850,29.40,34.1,30.60,21.2,33.9,29.000,29.3,28.30,26.400,13.2,29.10,30.7,34.800,27.6,38.2,25.8,29.8,28.1,26.70,26.4,0.412,4.870,2.300,-5.170,20.70,0.309,3.430,-4.280,-6.410,-4.33,...,8.37,-17.5,-17.5,8.37,101.0,-28.30,33.70,33.70,-28.30,130.0,4.54,-0.673,-0.673,4.54,106.0,6.59,19.9,19.9,6.59,155.0,2.67,14.00,14.00,2.67,140.0,-1.55,1.83,1.83,-1.55,106.0,-30.60,9.56,9.56,-30.60,92.6,-16.5,38.6,38.6,-16.5,0
3,30.8,32.3,30.7,27.70,25.3,0.159,-1.83,-1.80,-1.840,6.170,30.40,33.9,33.20,29.1,21.5,31.000,32.4,30.10,28.100,23.0,34.60,31.8,32.400,29.2,28.2,27.2,30.9,27.2,24.40,28.7,-0.573,1.520,3.060,0.955,-1.49,-4.200,2.150,0.705,-0.137,-6.67,...,-12.60,32.5,32.5,-12.60,111.0,8.19,-6.48,-6.48,8.19,130.0,-1.85,13.100,13.100,-1.85,55.9,-19.90,45.4,45.4,-19.90,133.0,5.89,-20.70,-20.70,5.89,147.0,-7.06,-17.30,-17.30,-7.06,166.0,-1.22,-15.20,-15.20,-1.22,144.0,-2.5,-34.5,-34.5,-2.5,0
4,-16.8,27.7,-867.0,9.68,22.0,-5.050,-2.95,-44.10,0.534,-7.610,9.89,28.7,-792.00,19.1,29.1,-37.800,29.7,-896.00,-0.066,22.5,-3.01,26.5,-847.000,17.6,21.8,-35.2,26.0,-930.0,2.51,14.6,47.700,-0.945,104.000,19.200,6.58,12.900,2.220,54.500,1.560,7.32,...,711.00,-311.0,-311.0,711.00,-631.0,646.00,-272.00,-272.00,646.00,-1060.0,816.00,-487.000,-487.000,816.00,-829.0,734.00,-380.0,-380.0,734.00,-549.0,576.00,-253.00,-253.00,576.00,-1100.0,767.00,-459.00,-459.00,767.00,-934.0,740.00,-392.00,-392.00,740.00,-748.0,664.0,-332.0,-332.0,664.0,2


In [12]:
import random
import numpy as np
from tqdm.notebook import tqdm

xxyy = pd.concat((x_data.copy(), y_data.copy()), axis=1).sample(frac=1).reset_index(drop=True)
xxyy.head()
xx_data = xxyy.loc[:,'# mean_0_a':'fft_749_b']
yy_data = xxyy.loc[:,'label']

std = xx_data.std()

for i in tqdm(range(len(xx_data))):
    for j in range(len(xx_data.columns)):
        if random.random() < 0.2:
            xx_data.values[i][j] += (random.random()-0.5)*std[i]*0.01


In [0]:
pd.concat((xx_data.copy(), yy_data.copy()), axis=1)[:1300].to_csv('train_data.csv', mode='w', index=False)
pd.concat((xx_data.copy(), yy_data.copy()), axis=1).loc[1300:,'# mean_0_a':'fft_749_b'].to_csv('test_data.csv', mode='w', index=False)

In [0]:
sample = pd.DataFrame([[i, i] for i in range(len(xx_data[1300:]))], columns=['Id',  'Category'])
sample.to_csv('submissions_sample.csv', mode='w', index=False)

In [0]:
real_test = pd.concat((xx_data.copy(), yy_data.copy()), axis=1).loc[1300:,'label']
real_test_df = pd.DataFrame([[i, r] for i, r in enumerate(real_test)], columns=['Id',  'Category'])
real_test_df.to_csv('real_submissions.csv', mode='w', index=False)